In [2]:
#Import all the required packages
import os
import datetime
import click
import matplotlib

import random as rn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from scipy import stats
from math import pi

from metrics import metrics
from quantileNetwork import QuantileNet, makeDataset, callNet

In [3]:
modelName = "raw_to_gen_full_v2_lrelu_1024batch_0.01LR_4cycles" #Name of the network to use
sampleNum=1000 #Number of samples to take for each prediction
pTMin=100 #Min pt for deltas
pTMax=120 #Max pt for deltas


#Set random number seeds
os.environ["PYTHONHASHSEED"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
np.random.seed(42)
rn.seed(12345)
tf.random.set_seed(3)    



#Extract data
data = np.load("mlDataPredictionsNewNorm.npy").T
rawRecoData = data[4:8,:]
recoData = data[8:12,:]
genData = data[12:16]

rawRecoData[0,:] = np.log(rawRecoData[0,:])
rawRecoData[3,:] = np.log(rawRecoData[3,:])
genData[0,:] = np.log(genData[0,:])
genData[3,:] = np.log(genData[3,:])

inputData=rawRecoData
outputData=genData

trainIn = inputData.T
trainOut = outputData.T
normInfoIn=[[0,1],[0,1],[0,1],[0,1]]
normInfoOut=[[0,1],[0,1],[0,1],[0,1]]
recoData=recoData.T
trainIn, testIn, trainOut, testOut = train_test_split(trainIn,
                                                    trainOut,
                                                    test_size=1/9,
                                                    random_state=42)
_,recoData,_,_ = train_test_split(recoData,
                                                    recoData,
                                                    test_size=1/9,
                                                    random_state=42)
recoData=recoData.T

testIn = testIn
trainIn = trainIn
testOut = testOut
trainOut = trainOut

In [4]:
#Normalization 
for x in [0,1,3]:
    normInfoIn[x]=[np.mean(testIn[:,x]),np.std(testIn[:,x])]
    normInfoOut[x]=[np.mean(testOut[:,x]),np.std(testOut[:,x])]
    testIn[:,x]=(testIn[:,x]-np.mean(testIn[:,x]))/(np.std(testIn[:,x]))
    testOut[:,x]=(testOut[:,x]-np.mean(testOut[:,x]))/(np.std(testOut[:,x]))

currentTestIn=[]
currentTestOut=[]
currentRecoCorrected=[]

#Filter inputs
for x in range(len(testIn)):
    #Gen delta
    #if(False or (np.exp(testOut[x,0]*normInfoOut[0][1]+normInfoOut[0][0])<pTMax and np.exp(testOut[x,0]*normInfoOut[0][1]+normInfoOut[0][0]))>pTMin):
    
    #Raw reco delta (eta)
    #if(False or (testIn[x,1]*normInfoIn[1][1]+normInfoIn[1][0]<-2.7 or testIn[x,1]*normInfoIn[1][1]+normInfoIn[1][0]>2.7)):
    
    #Raw reco delta (phi) 
    #if(False or (testIn[x,2]*normInfoIn[2][1]+normInfoIn[2][0]>pi/8 and testIn[x,2]*normInfoIn[2][1]+normInfoIn[2][0]<pi/4)):
    
    #Raw reco delta (pt)
    #if(False or (np.exp(testIn[x,0]*normInfoIn[0][1]+normInfoIn[0][0])<pTMax and np.exp(testIn[x,0]*normInfoIn[0][1]+normInfoIn[0][0]))>pTMin):
    if(True):
        currentTestIn.append(testIn[x,:])
        currentTestOut.append(testOut[x,:])
        currentRecoCorrected.append(recoData[:,x])
#Datasets
testIn = np.array(currentTestIn)
testOut = np.array(currentTestOut)
recoData = np.array(currentRecoCorrected)
model = QuantileNet() #Used to load network
testIn=tf.transpose(testIn[:,:])
testIn=tf.cast(testIn, tf.float32)
newModel = tf.keras.models.load_model(modelName, custom_objects={"QuantileNet": QuantileNet, "loss": model.loss})

#out=testCall(newModel,sampleNum,tf.cast(testIn, tf.float32),testIn.shape[1],testIn.shape[0],4,1+0*4)
#Call the network
out=callNet(newModel, #Network
            sampleNum, #Number of samples
            tf.cast(testIn, tf.float32), #Input
            testIn.shape[1], #Number of examples (batch size)
            testIn.shape[0], #4d input
            4) #4d output


(23391, 4)
(23391, 4)
23391 tf.Tensor(
[[ 0.21008128  0.21008128  0.21008128 ...  0.21008128  0.21008128
   0.21008128]
 [-1.2392883  -1.2392883  -1.2392883  ... -1.2392883  -1.2392883
  -1.2392883 ]
 [ 2.59977     2.59977     2.59977    ...  2.59977     2.59977
   2.59977   ]
 ...
 [ 0.         -0.         -0.         ... -0.         -0.
   0.        ]
 [-0.          0.         -0.         ... -0.          0.
  -0.        ]
 [-2.701732   -2.2525234   0.6665032  ...  1.4977255  -1.8382344
   0.97242594]], shape=(12, 1000), dtype=float32)
finalInput (23391000, 12)


AttributeError: in user code:

    /home/DAVIDSON/brkronheim/phy3/Honors/quantileNetwork.py:280 predict_inner  *
        for layer in quantileObject.netLayers:

    AttributeError: 'QuantileNet' object has no attribute 'netLayers'


In [ ]:
output=np.array(out)
#Extract predictions
pt = np.reshape(output[:,:,0],-1)
eta = np.reshape(output[:,:,1],-1)
phi  = np.reshape(output[:,:,2],-1)
e  = np.reshape(output[:,:,3],-1)

#Some calculations
dPhi2Pred = np.minimum(np.square(np.repeat(phiGen,sampleNum)-phiPred), np.square(phiPred-(np.repeat(phiGen,sampleNum))))
dPhi2Reco = np.minimum(np.square(phiGen-phiReco), np.square(phiReco-phiGen))
dPhi2Raw = np.minimum(np.square(phiGen-phiRawReco), np.square(phiRawReco-phiGen))


dEta2Pred = np.square(np.repeat(etaGen,sampleNum)-etaPredicted)
dEta2Reco = np.square(etaGen-etaReco)
dEta2Raw = np.square(etaGen-etaRawReco)

dRPred = np.sqrt(dEta2Pred+dPhi2Pred)
dRReco = np.sqrt(dEta2Reco+dPhi2Reco)
dRRaw = np.sqrt(dEta2Raw+dPhi2Raw)


ptRatioPred = np.abs(ptPredicted)/np.repeat(ptGen, sampleNum)
ptRatioReco = np.abs(ptReco)/ptGen
ptRatioRaw = np.abs(ptRawReco)/ptGen
ptPredicted = np.exp(pt*normInfoOut[0][1]+normInfoOut[0][0])

ptReco = recoData[:,0]
ptGen = np.exp(testOut[:,0]*normInfoOut[0][1]+normInfoOut[0][0])
ptRawReco = np.exp(testIn[0,:]*normInfoIn[0][1]+normInfoIn[0][0])

etaPredicted = np.array(eta*normInfoOut[1][1]+normInfoOut[1][0])
etaReco = recoData[:,1]
etaGen = testOut[:,1]*normInfoOut[1][1]+normInfoOut[1][0]
etaRawReco = np.array(testIn[1,:]*normInfoIn[1][1]+normInfoIn[1][0])

phiPred = np.array(phi*normInfoOut[2][1]+normInfoOut[2][0])%(2*pi)
phiReco = np.array(recoData[:,2])%(2*pi)
phiGen = np.array(testOut[:,2]*normInfoOut[2][1]+normInfoOut[2][0])%(2*pi)
phiRawReco = np.array(testIn[2,:]*normInfoIn[2][1]+normInfoIn[2][0])%(2*pi)

ePred = np.exp(e*normInfoOut[3][1]+normInfoOut[3][0])
eReco = recoData[:,3]
eGen = np.exp(testOut[:,3]*normInfoOut[3][1]+normInfoOut[3][0])
eRawReco = np.exp(testIn[3,:]*normInfoIn[3][1]+normInfoIn[3][0])

dPhi2Pred = np.minimum(np.square(np.repeat(phiGen,sampleNum)-phiPred), np.square(phiPred-(np.repeat(phiGen,sampleNum))))
dPhi2Reco = np.minimum(np.square(phiGen-phiReco), np.square(phiReco-phiGen))
dPhi2Raw = np.minimum(np.square(phiGen-phiRawReco), np.square(phiRawReco-phiGen))


dEta2Pred = np.square(np.repeat(etaGen,sampleNum)-etaPredicted)
dEta2Reco = np.square(etaGen-etaReco)
dEta2Raw = np.square(etaGen-etaRawReco)

dRPred = np.sqrt(dEta2Pred+dPhi2Pred)
dRReco = np.sqrt(dEta2Reco+dPhi2Reco)
dRRaw = np.sqrt(dEta2Raw+dPhi2Raw)


ptRatioPred = np.abs(ptPredicted)/np.repeat(ptGen, sampleNum)
ptRatioReco = np.abs(ptReco)/ptGen
ptRatioRaw = np.abs(ptRawReco)/ptGen

In [ ]:
import matplotlib
font = {'family' : 'serif',
        'size'   : 12}

matplotlib.rc('font', **font)



#Make some plots
bins=20
plt.figure()
plt.hist(ePred**2-ptPredicted**2/(1-np.tanh(etaPredicted)**2), bins=bins, label="predicted", histtype="step",  range=(-500,500), color="r", weights=ePred*0+1/1000)
plt.hist(eGen**2-ptGen**2/(1-np.tanh(etaGen)**2), bins=bins,label="true", histtype="step", range=(-500,500), color="k")
plt.xlabel("Mass Squared (GeV^2)")
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


plt.figure(figsize=(6,4))
plt.hist(np.sqrt(dPhi2Raw), range=(0,5), bins=bins, label="raw reco", histtype="step", color="b")
plt.hist(np.sqrt(dPhi2Reco), range=(0,5), bins=bins, label="reco", histtype="step", color="orange")
plt.hist(np.sqrt(dPhi2Pred), range=(0,5), bins=bins, label="predicted", histtype="step", color="red", weights=np.ones(dEta2Pred.shape)/sampleNum)
plt.legend()
plt.title("dPhi distributions")
plt.tight_layout()
plt.show()

plt.figure()
plt.hist(np.sqrt(dEta2Raw), range=(0,5), bins=bins, label="raw reco", histtype="step", color="b")
plt.hist(np.sqrt(dEta2Reco), range=(0,5), bins=bins, label="reco", histtype="step", color="orange")
plt.hist(np.sqrt(dEta2Pred), range=(0,5), bins=bins, label="predicted", histtype="step", color="red", weights=np.ones(dEta2Pred.shape)/sampleNum)
plt.legend()
plt.title("dEta distributions")
plt.tight_layout()
plt.show()


plt.figure()

plt.hist(dRRaw, range=(0,5), bins=bins, label="raw reco", histtype="step", color="b")
plt.hist(dRReco, range=(0,5), bins=bins, label="reco", histtype="step", color="orange")
plt.hist(dRPred, range=(0,5), bins=bins, label="predicted", histtype="step", color="red", weights=np.ones(dEta2Pred.shape)/sampleNum)

plt.legend()
plt.title("dR distributions")
plt.tight_layout()
plt.show()

plt.figure()
plt.hist(ptRatioRaw, range=(0,2), bins=bins, label="raw reco", histtype="step", color="b")
plt.hist(ptRatioReco, range=(0,2), bins=bins, label="reco", histtype="step", color="orange")
plt.hist(ptRatioPred, range=(0,2), bins=bins, label="predicted", histtype="step", color="red", weights=np.ones(dEta2Pred.shape)/sampleNum)
plt.legend()
plt.title("pT Ratios")
plt.tight_layout()
plt.show()



plt.figure()
plt.hist(ptRawReco, range=(0,300), bins=bins, label="raw reco", histtype="step", color="b")
plt.hist(ptReco, range=(0,300), bins=bins, label="reco", histtype="step", color="orange")
plt.hist(ptPredicted, range=(0,300), bins=bins, label="predicted", histtype="step", weights=np.ones(dEta2Pred.shape)/sampleNum, color="r")
plt.hist(ptGen, range=(0,300), bins=bins, label="gen", histtype="step", color="g")
plt.xlabel("pT (GeV)")
plt.legend()
plt.title("pT Distribution")
plt.tight_layout()
plt.show()


In [ ]:
n=len(ptGen)
m=n
en = m * n / (m + n)
stat, p = stats.ks_2samp(ptPredicted,ptGen)
p = stats.kstwo.sf(stat, np.round(en))
print("pt same prob", p)
stat, p = stats.ks_2samp(etaPredicted,etaGen)
p = stats.kstwo.sf(stat, np.round(en))
print("eta same prob", p)
stat, p = stats.ks_2samp(phiPred,phiGen)
p = stats.kstwo.sf(stat, np.round(en))
print("phi same prob", p)
stat, p = stats.ks_2samp(ePred,eGen)
p = stats.kstwo.sf(stat, np.round(en))
print("e same prob", p)